# Обучение языковой модели с помощью LSTM

Цель: построить генеративную языковую модель, которая по началу/теме предложения сможет генерировать осмысленный текст.
Основной задачей является экспериментирование с разными схемами моделей и параметрами, грамотная оценка качества модели через перплексию.

---

## 0. Зависимости.
Стандартные библиотеки для работы с текстом. Настройка MLflow для логирования экспериментов.

In [37]:
!pip install datasets
!pip install mlflow

In [38]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List
import mlflow
import seaborn


#Секрет - адрес mlflow-сервера
from kaggle_secrets import UserSecretsClient

secret_value = UserSecretsClient().get_secret("mlflow_url")
mlflow.set_tracking_uri(secret_value)
seaborn.set(palette='summer')


In [39]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Подготовка данных.
Используется датасет imdb. В нем хранятся отзывы о фильмах с сайта imdb.

In [40]:
dataset = load_dataset('imdb')

  0%|          | 0/3 [00:00<?, ?it/s]

## 2. Препроцессинг данных и создание словаря
Далее необходмо получить словарь или же просто set строк. Что необходимо сделать:

- Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции sent_tokenize из бибилиотеки nltk. Каждое отдельное предложение будет одним тренировочным примером.
- Оставить только те предложения, в которых меньше word_threshold слов.
- Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предложения на отдельные слова удобно использовать функцию word_tokenize.
- Создать объект vocab класса set, положить в него служебные токены ```'<unk>', '<bos>', '<eos>', '<pad>'``` и vocab_size самых частовстречающихся слов.

In [41]:
# Первые два пункта
def filter_on_length(sentence: str, threshold=32):
    # Фильтруем по длине, отбрасывая длинные.
    word_count = len(word_tokenize(sentence))
    if word_count > threshold:
        return False
    return True

def get_sentences(data: Dataset, threshold=32) ->list:
    # Преобразуем датасет в список с предложениями
    sentences = []
    for review in tqdm(data["text"]):
        sentences +=[x.lower() for x in sent_tokenize(review) if filter_on_length(x, threshold)]
    return sentences

In [42]:
sentence_list = get_sentences(dataset["train"])
print(len(sentence_list))

  0%|          | 0/25000 [00:00<?, ?it/s]

202606


In [43]:
# Создание словаря
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
vocab_size = 40000

def count_words(sentences, size):
    word_dict = {}
    for sentence in tqdm(sentences):
        for word in Counter(word_tokenize(sentence)):
            if word in word_dict:
                word_dict[word] += 1
            else:
                word_dict[word] = 1
    word_dict = [key for key, _ in sorted(word_dict.items(), key=lambda item: item[1], reverse=True)]
    return word_dict[:size]

In [44]:
vocab.update(count_words(sentence_list, 40000))
assert '<unk>' in vocab
assert '<bos>' in vocab 
assert '<eos>' in vocab 
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4
print(len(vocab))

  0%|          | 0/202606 [00:00<?, ?it/s]

40004


## 3. Dataset и лоадеры.
Датасет с методом `__getitem__` и затем train- val- loaders

In [45]:
word2ind = {word: i for i, word in enumerate(vocab)}
ind2word = {i: word for word, i in word2ind.items()}

class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        # Маркеры неизв. слова, начала и конца предложения, паддинга.
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        # Предложения приведены к нижнему регистру, 
        # но не очищены от стоп-слов и пунктуации.
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            word2ind.get(word, self.unk_id) for word in word_tokenize(self.data[idx])
            ] 
        tokenized_sentence += [self.eos_id]

        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)
    
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    # Паддинг: нужно выдавать entry одинаковой длины, соотв. заполняем нехват. строки
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)
    
    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [46]:
train_sentences, eval_sentences = train_test_split(sentence_list, test_size=0.2)
eval_sentences, test_sentences = train_test_split(sentence_list, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 128

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

## 4. Обучение и архитектура модели
---
Для обучения и логирования экспериментов понадобится функции train, evaluate, generate.
Train - стандартный, с логированием точности и лосса на обучающей и валидационной выборке + построением графиков.
Точность - из evaluate - перплексия = exp(CrossEntropy).

Экспериментальная часть подразумевает как перебор гиперпараметров, так и разные конфигурации моделей и нормализации.

In [47]:
def fit(train_dataloader, model, criterion, optimizer, epoch):
    epoch_losses = []
    model.train()
    for batch in tqdm(train_dataloader, desc=f'Training epoch {epoch}:'):
        optimizer.zero_grad()
        logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
        loss = criterion(
            logits, batch['target_ids'].flatten())
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    return epoch_losses

def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())
    
    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

def train(train_loader, eval_loader, model, criterion, optimizer, epochs=7):
    losses = []
    perplexities = []
    for epoch in range(epochs):
        epoch_losses = fit(train_loader, model, criterion, optimizer, epoch)
        mlflow.log_metric("train loss", sum(epoch_losses) / len(epoch_losses))
        losses.append(sum(epoch_losses) / len(epoch_losses))
        
        perp = evaluate(model, criterion, eval_loader)
        mlflow.log_metric("val perplexity", perp)
        perplexities.append(perp)
    return losses, perplexities

In [48]:
def plot_results(losses, perplexity):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].plot(np.arange(len(losses)), losses)
    axes[0].set_title('Losses')
    axes[0].set_xlabel("epoch")
    axes[1].plot(np.arange(len(perplexity)), perplexity)
    axes[1].set_title('Accuracy on val.')
    plt.show()

In [49]:
class WordLM(nn.Module):
    def __init__(self, 
                 hidden_dim: int, 
                 vocab_size: int,
                 num_layers=1, 
                 drop_rate=0.1,
                 batches=False,
                 rnn_type="rnn"
                 ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        if rnn_type == "rnn":
            self.rnn = nn.RNN(hidden_dim, 
                              hidden_dim, 
                              num_layers=num_layers, 
                              batch_first=True)
        elif rnn_type == "gru":
            self.rnn = nn.GRU(hidden_dim, 
                              hidden_dim, 
                              num_layers=num_layers, 
                              batch_first=True)
        else:
            self.rnn = nn.LSTM(hidden_dim, 
                              hidden_dim, 
                              num_layers=num_layers, 
                              batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]
        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.non_lin(output))  # [batch_size, seq_len, vocab_size]

        return projection

## 5. Baseline.


In [50]:
"""criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
first_model = WordLM(hidden_dim=128, vocab_size=len(vocab)).to(device)
optimizer = torch.optim.Adam(first_model.parameters())
losses, perp = train(train_dataloader, eval_dataloader, first_model, criterion, optimizer, epochs=10)"""

"criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])\nfirst_model = WordLM(hidden_dim=128, vocab_size=len(vocab)).to(device)\noptimizer = torch.optim.Adam(first_model.parameters())\nlosses, perp = train(train_dataloader, eval_dataloader, first_model, criterion, optimizer, epochs=10)"

In [51]:
#plot_results(losses, perp)
#print(perp)

In [52]:
def generate(model, input_seq:str, length=30) -> str:
    device = 'cpu'
    model = model.to(device)
    input_ids = [word2ind['<bos>']] + [
        word2ind.get(word, word2ind['<unk>']) for word in word_tokenize(input_seq)]
    input_ids = torch.LongTensor(input_ids).to(device)
    
    model.eval()
    with torch.no_grad():
        for i in range(length):
            next_word_distribution = model(input_ids)[-1]
            # print(next_word_distribution.squeeze())
            next_word = next_word_distribution.squeeze().argmax()
            if next_word.item() == word2ind['<eos>']:
                break
            input_ids = torch.cat([input_ids, next_word.unsqueeze(0)])
    
    words = ' '.join([ind2word[idx.item()] for idx in input_ids])

    return words

In [53]:
#print(generate(first_model, input_seq='acting and directing'))


## 6. Эксперименты.
Структура экспериментов предполагается следующей:
```
[EXP-1] - [Architecture]
			-[LSTM:RNN:GRU]
            -[bidirectional]
			-[num_layers 1:5:10]
[EXP-2] - [Dropout 0.1:0.5]	
[EXP-3] - [Batchnorm]
[EXP-4] - [Hyperparams]
---
[EXP-5] - final model, long train
```
Тесты проводятся со сроком 7 эпох. Затем лучшая модель+параметры обучаются 20 эпох

In [54]:
def start(model_type, 
          num_layers, 
          hidden_dim, 
          drop_rate, 
          batches,
          epochs):
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
    model = WordLM(hidden_dim=hidden_dim, 
                         vocab_size=len(vocab), 
                         num_layers=num_layers, 
                         drop_rate=drop_rate,
                         batches=batches,
                         rnn_type=model_type).to(device)
    optimizer = torch.optim.Adam(first_model.parameters())
    return model, train(train_dataloader, eval_dataloader, model, criterion, optimizer, epochs)


In [55]:
mlflow.set_experiment(experiment_name=f"LM: EXP-1 [Architecture].")
models = ["rnn", "gru", "lstm"]
params = {
        "num_layers": 1,
        "epochs": 7,
        "hidden_dim": 128,
        "drop_rate": 0.1,
        "batches": False
    }
for model_type in models:
    with mlflow.start_run():
        mlflow.log_params(params)
        mlflow.log_param("rnn_type", model_type)
        model, losses, perps = start(model_type=model_type, **params)
        print(f"Plots for {model_type}")
        plot_results(losses, perps)
        criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
        perplexity = evaluate(model, criterion, eval_dataloader)
        mlflow.log_metric("perplexity", perplexity)
        print(f"Final perplexity: {perplexity}")
        

Training epoch 0::   0%|          | 0/1267 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/1267 [00:00<?, ?it/s]

KeyboardInterrupt: 